In [1]:
from pyspark import SparkConf, SparkContext,SQLContext  
from pyspark.sql import SparkSession   
from pyspark.ml.feature import Word2Vec,CountVectorizer  
from pyspark.ml.clustering import LDA, LDAModel  
from pyspark.sql.functions import col, udf, split 
from pyspark.sql.types import IntegerType,ArrayType,StringType,StructType, StructField
import pylab as pl  

In [2]:
def to_word(termIndices):
  words = []  
  for termID in termIndices:
    words.append(vocab_broadcast.value[termID])      
  return words

In [3]:
#Load your document dataframe here
#================your code here==================
Schema = StructType([
    StructField("twittes", StringType(), True),
])
spark = SparkSession \
    .builder \
    .getOrCreate()
df = spark.read.option("inferSchema", "True").schema(Schema).csv("gs://eecs6893_data/notebooks/jupyter/EECS_6893_Big_Data_Analysis/HW2/stream_data.csv")
spark_df = df.select(split(col("twittes")," ")).withColumnRenamed("split(twittes,  , -1)","words")
#==================================================
spark_df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/23 15:35:07 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/23 15:35:07 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/23 15:35:07 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/23 15:35:07 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


+--------------------+
|               words|
+--------------------+
|[I, absolutely, A...|
|[Java, Vs, Python...|
|[voulu, un, grec,...|
|[Pareil, Il, pris...|
|[Music, Academy, ...|
|[Tarps,, tents,, ...|
|[voulu, un, grec,...|
|[We, drive, effic...|
|[Check, out, my, ...|
|[Hey,, nice, bone...|
|[lembro, como, so...|
|[WHO, WITH, A, DE...|
|[@Tina69911364, @...|
|[alguem, cria, um...|
|[@Neptvn08, Comme...|
|[une, dinguerie, ...|
|[Y, a, une, gross...|
|[Je, te, cache, p...|
|[@JAPANFESS, seta...|
|[Femme, rechercha...|
+--------------------+
only showing top 20 rows



In [4]:
#CountVectorizer
#================your code here==================
# word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="words", outputCol="word2Vec")
# model = word2Vec.fit(spark_df)
# cvResult = model.transform(spark_df)
    
cv = CountVectorizer(inputCol="words", outputCol="features", minDF=30)
model = cv.fit(spark_df)
cvResult = model.transform(spark_df)
#==================================================

In [5]:
#train LDA model, cluster the documents into 10 topics 
#================your code here==================
lda = LDA(featuresCol="features",maxIter=1000, k=10, learningDecay=0.45, optimizer='online', topicDistributionCol='topicDistribution')
ldaModel = lda.fit(cvResult)
#==================================================

In [6]:
transformed = ldaModel.transform(cvResult).select("topicDistribution")  
#show the weight of every topic Distribution 
transformed.show(truncate=False)  

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topicDistribution                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.009726653482341288,0.003981236916849474,0.00169054864452851,0.014606748948324193,0.01900827575610118,0.9355745024303354,0.0016907908124763852,0.010340145272679279,0.0016905488145869908,0.0016905489217772613] |
|[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]                                                                                                      

In [7]:
#The higher ll is, the lower lp is, the better model is.
ll = ldaModel.logLikelihood(cvResult)  
lp = ldaModel.logPerplexity(cvResult)
print("ll: ", ll)
print("lp: ", lp)

ll:  -9299.719539892056
lp:  3.308331390925669


In [8]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())+ " words):")
topics = ldaModel.topicsMatrix()
print(topics)

Learned topics (as distributions over vocab of 40 words):
DenseMatrix([[3.64354276e+01, 1.00004720e-01, 1.00000000e-01, 1.00013217e-01,
              3.27534104e+01, 1.48932363e+02, 1.00000000e-01, 1.00016255e-01,
              1.00000000e-01, 1.00000000e-01],
             [1.00000000e-01, 1.00004073e-01, 1.00000000e-01, 1.00000090e-01,
              1.00000213e-01, 1.47440521e+02, 1.00000000e-01, 4.11656658e+01,
              1.00000000e-01, 1.00000000e-01],
             [1.90478278e+01, 1.00006865e-01, 1.00000000e-01, 8.17683588e+01,
              7.85089572e+01, 1.00000050e-01, 1.00000000e-01, 1.00000140e-01,
              1.00000000e-01, 1.00000000e-01],
             [1.00000700e-01, 1.00000000e-01, 1.00000000e-01, 1.00000145e-01,
              1.00000460e-01, 9.11496324e+01, 1.00000000e-01, 4.70844861e+01,
              1.00000000e-01, 1.00000000e-01],
             [1.00007944e-01, 1.00017998e-01, 1.00000000e-01, 1.00000000e-01,
              1.00002517e-01, 1.17613057e+02, 1.0000